In [18]:
kappa_r = 1.5179
theta_r = 0.0582
sigma_r = 0.0922

kappa_d = 1.5472
theta_d = 0.0020
sigma_d = 0.1647

In [6]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import pandas as pd
import numpy as np


# In[ ]:


file_name = '../data/usdrub_data.csv'


# In[ ]:


df_ret = pd.read_csv(file_name)


# In[ ]:


col_name = 'RUB=X' if 'RUB=X' in df_ret.columns else df_ret.columns[1]


# In[ ]:


df_ret['Date'] = pd.to_datetime(df_ret['Date'])
df_ret = df_ret.sort_values('Date').set_index('Date')


# In[ ]:


df_ret = df_ret['2019-01-01':'2021-12-10']

In [12]:

file_name = '../data/SOFR.csv'


# In[20]:


df_usd = pd.read_csv(file_name)


# In[21]:


df_usd['Date'] = pd.to_datetime(df_usd['observation_date'])
df_usd = df_usd.sort_values('Date').set_index('Date')


# In[22]:


df_usd = df_usd['2019-01-01':'2021-12-10']


# In[23]:


df_usd['r'] = df_usd['SOFR'].values / 100

r = df_usd['r'].dropna().values

dt = 1 / 252

In [11]:
file_name = '../data/RC_F11_01_2010_T10_12_2021.csv'


# In[4]:


df_rub = pd.read_csv(file_name, delimiter=',', decimal='.')


# In[5]:


df_rub.columns = df_rub.columns.str.strip()


# In[6]:


df_rub["Date"] = pd.to_datetime(df_rub["DT"], format="%m/%d/%Y")
df_rub = df_rub.sort_values('Date').set_index('Date')


# In[7]:


df_rub = df_rub['2019-01-01':'2021-12-10']

In [4]:
import model

sigma_fx, sigma_fx_daily, log_ret = model.estimate_fx_vol_from_series(
    df[col_name],
    dt_years=1/252
)

In [5]:
sigma_fx

0.1290819405516298

In [16]:
df_usd.shape, df_rub.shape, df_usd.shape

((769, 3), (719, 11), (769, 3))

In [19]:
dr_rub = np.diff(df_rub["ruo"].values / 100)
dr_usd = np.diff(df_usd["SOFR"].values / 100)
log_ret_fx = np.diff(np.log(df_ret["RUB=X"].values))

# corr_matrix = model.compute_corr_matrix(dr_rub, dr_usd, log_ret_fx)


corr_data = [
    [1.0000, 0.0382, 0.0098],
    [0.0382, 1.0000, -0.0177],
    [0.0098, -0.0177, 1.0000]
]
corr_matrix = np.array(corr_data)

In [20]:
r0_rub = df_rub["ruo"].values[0] / 100
r0_usd = df_usd["SOFR"].values[0] / 100
s0_fx = df_ret["RUB=X"].values[0]

rub_paths, usd_paths, fx_paths, disc_int = model.simulate_market_paths(
    N_sim=10000,
    N_steps=252,
    T_years=1.0,
    r0_rub=r0_rub,
    r0_usd=r0_usd,
    s0_fx=s0_fx,
    cir_rub_params=(kappa_r, theta_r, sigma_r),
    cir_usd_params=(kappa_d, theta_d, sigma_d),
    fx_vol=sigma_fx,
    corr_matrix=corr_matrix,
)


In [23]:
import datetime

fair, se = model.price_range_accrual(
    notional=1_000_000,
    lower_barrier=70.0,
    upper_barrier=80.0,
    start_date=datetime.date(2021, 12, 10),
    end_date=datetime.date(2022, 12, 10),
    r0_rub=r0_rub,
    r0_usd=r0_usd,
    s0_fx=s0_fx,
    cir_rub_params=(kappa_r, theta_r, sigma_r),
    cir_usd_params=(kappa_d, theta_d, sigma_d),
    fx_vol=sigma_fx,
    corr_matrix=corr_matrix,
    N_sim=10000,
    steps_per_year=252,
)
print(fair, se)


0.0 0.0
